<a href="https://colab.research.google.com/github/sugi-san/3d-movie/blob/master/3d_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#テーマ：３D動画
・扱える写真の種類は、**jpg**と**png**です。ピクセルサイズはHD（1280×720）〜フルHD（1920×1080）くらいが目安\
・1〜11番までの文章が並んでいて、番号の前に何もないのは「手順」、番号の前に黒丸があるのは「プログラム」です

## 1.自分のPCからマシンへ接続
**右上にある接続ボタンをクリック　→　接続完了で緑色のチェックマーク表示**

In [ ]:
#@title 2.マシンにソフトをインストール
# ライブラリーのインストール
!pip install chainer
!pip install moviepy
!pip install gevent

# githubからコードを取得
!git clone https://github.com/sugi-san/3d-movie.git
%cd /content/3d-movie

# ディレクトリの作成
!mkdir output

# ライブラリーのインポート
import chainer
import os
chainer.print_runtime_info()
os.environ['CUDA_HOME'] = "/usr/local/cuda"

# 学習済みモデルのダウンロード
!wget -O ./models/disparity-estimation.pytorch http://content.sniklaus.com/kenburns/network-disparity.pytorch
!wget -O ./models/disparity-refinement.pytorch http://content.sniklaus.com/kenburns/network-refinement.pytorch
!wget -O ./models/pointcloud-inpainting.pytorch http://content.sniklaus.com/kenburns/network-inpainting.pytorch

# パラメータのダウンロード
import torch
import torchvision
import cupy
exec(open('./common.py', 'r').read())
exec(open('./models/disparity-estimation.py', 'r').read())
exec(open('./models/disparity-adjustment.py', 'r').read())
exec(open('./models/disparity-refinement.py', 'r').read())
exec(open('./models/pointcloud-inpainting.py', 'r').read())

## 3.左のウインドウを開く
**左端にあるフォルダーマークをクリックし、3d_movieフォルダーの先頭をクリックして開く**\
＊左のウインドウは実際と表示にタイムラグがあるので、必要に応じて左上にある更新ボタンを押す


**----------- サンプル画像でやってみる -------------**



### 4.サンプル画像の確認
**inputフォルダーの先端をクリックして開き、中にある画像ファイルをダブルクリック。確認後×で戻る**\
\
**shift : （基準：100）視点と写真の中心点との差**\
**distance : （基準：1.25）視点の前方への移動距離**

In [ ]:
#@title 5.動画の作成（inputフォルダーにある画像から選ぶ）
import torch
import torchvision
import base64
import cupy
import cv2
import flask
import getopt
import gevent
import gevent.pywsgi
import glob
import h5py
import io
import math
import moviepy
import moviepy.editor
import numpy
import os
import random
import re
import scipy
import scipy.io
import shutil
import sys
import tempfile
import time
import urllib
import zipfile

assert(int(str('').join(torch.__version__.split('.')[0:2])) >= 12) # requires at least pytorch version 1.2.0
torch.set_grad_enabled(False) # make sure to not compute gradients for computational performance
torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance
objCommon = {}

input = '02.jpg' #@param {type:"string"}
arguments_strIn = './input/'+input
arguments_strOut = './output/movie.mp4'
 
shift =  100#@param {type:"slider", min:0, max:200, step:10}
distance = 1.25  #@param {type:"slider", min:1.25, max:2, step:0.05}
end = 4 
step = int(160*distance - 20)

##########################################################
npyImage = cv2.imread(filename=arguments_strIn, flags=cv2.IMREAD_COLOR)

intWidth = npyImage.shape[1]
intHeight = npyImage.shape[0]

fltRatio = float(intWidth) / float(intHeight)

intWidth = min(int(1024 * fltRatio), 1024)
intHeight = min(int(1024 / fltRatio), 1024)

# ----------
if not intWidth % 2 == 0:
  intWidth +=1

if not intHeight % 2 ==0:
  intHeight +=1
# ----------

npyImage = cv2.resize(src=npyImage, dsize=(intWidth, intHeight), fx=0.0, fy=0.0, interpolation=cv2.INTER_AREA)

process_load(npyImage, {})

objFrom = {
		'fltCenterU': intWidth / 2.0,
		'fltCenterV': intHeight / 2.0,
		'intCropWidth': int(math.floor(0.97 * intWidth)),
		'intCropHeight': int(math.floor(0.97 * intHeight))
}

objTo = process_autozoom({
		'fltShift': shift,  
		'fltZoom': distance, 
		'objFrom': objFrom
})

npyResult = process_kenburns({
		'fltSteps': numpy.linspace(0.0, end, step).tolist(),
		'objFrom': objFrom,
		'objTo': objTo,
		'boolInpaint': True
})

moviepy.editor.ImageSequenceClip(sequence=[ npyFrame[:, :, ::-1] for npyFrame in npyResult + list(reversed(npyResult))[1:] ], fps=25).write_videofile(arguments_strOut)


In [ ]:
#@title 6.動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output/movie.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="70%" height="70%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

### 7.動画のダウンロード
**outputフォルダーの先端をクリックして開き、中にあるmovie.mp4の右端の３つの点をクリックし、ダウンロードを選択**\
＊ダウンロードしたら適当な名前に変更して下さい

**---------- 自分の画像でやってみる -------------**

### 8.自分の写真をアップロード
**inputフォルダーの右端の３つの点をクリックし、アップロードを選択**\
\
**shift : （基準：100）視点と写真の中心点との差**\
**distance : （基準：1.25）視点の前方への移動距離**

In [ ]:
#@title 9.動画の作成（inputフォルダーにある画像から選ぶ）
import torch
import torchvision
import base64
import cupy
import cv2
import flask
import getopt
import gevent
import gevent.pywsgi
import glob
import h5py
import io
import math
import moviepy
import moviepy.editor
import numpy
import os
import random
import re
import scipy
import scipy.io
import shutil
import sys
import tempfile
import time
import urllib
import zipfile

assert(int(str('').join(torch.__version__.split('.')[0:2])) >= 12) # requires at least pytorch version 1.2.0
torch.set_grad_enabled(False) # make sure to not compute gradients for computational performance
torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance
objCommon = {}

input = '' #@param {type:"string"}
arguments_strIn = './input/'+input
arguments_strOut = './output/movie.mp4'
 
shift =  100#@param {type:"slider", min:0, max:200, step:10}
distance = 1.25  #@param {type:"slider", min:1.25, max:2, step:0.05}
end = 4 
step = int(160*distance - 20)

##########################################################
npyImage = cv2.imread(filename=arguments_strIn, flags=cv2.IMREAD_COLOR)

intWidth = npyImage.shape[1]
intHeight = npyImage.shape[0]

fltRatio = float(intWidth) / float(intHeight)

intWidth = min(int(1024 * fltRatio), 1024)
intHeight = min(int(1024 / fltRatio), 1024)

# ----------
if not intWidth % 2 == 0:
  intWidth +=1

if not intHeight % 2 ==0:
  intHeight +=1
# ----------

npyImage = cv2.resize(src=npyImage, dsize=(intWidth, intHeight), fx=0.0, fy=0.0, interpolation=cv2.INTER_AREA)

process_load(npyImage, {})

objFrom = {
		'fltCenterU': intWidth / 2.0,
		'fltCenterV': intHeight / 2.0,
		'intCropWidth': int(math.floor(0.97 * intWidth)),
		'intCropHeight': int(math.floor(0.97 * intHeight))
}

objTo = process_autozoom({
		'fltShift': shift,  
		'fltZoom': distance, 
		'objFrom': objFrom
})

npyResult = process_kenburns({
		'fltSteps': numpy.linspace(0.0, end, step).tolist(),
		'objFrom': objFrom,
		'objTo': objTo,
		'boolInpaint': True
})

moviepy.editor.ImageSequenceClip(sequence=[ npyFrame[:, :, ::-1] for npyFrame in npyResult + list(reversed(npyResult))[1:] ], fps=25).write_videofile(arguments_strOut)


In [ ]:
#@title 10.mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output/movie.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="70%" height="70%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

### 11.動画のダウンロード
**outputフォルダーの中にあるmovie.mp4の右端の３つの点をクリックし、ダウンロードを選択**\
＊ダウンロードしたら適当な名前に変更して下さい

### 【備考】
・このノートを実行して閉じてから、**すぐ再度実行**する場合は、前の記憶が不完全に残っていて上手く動かないことがあります。その場合は、右上にある**「ランタイム／ランタイムを出荷時設定にリセット」**をクリックしてから、**１番**から操作を行います。\
・長時間使用するとgoogleから使用制限を受ける場合がありますが、12h〜24h程度経つと使用制限が解除されます。\